In [1]:
from pyiron import Project
import numpy as np

In [2]:
pr = Project("water_interactive")
pr.remove_jobs_silently()

In [3]:
dx = 0.7
cell = np.eye(3) * 10
r_O = [0, 0, 0]
r_H1 = [dx, dx, 0]
r_H2 = [-dx, dx, 0]
water = pr.create_atoms(elements=['H', 'H', 'O'],
                        positions=[r_H1, r_H2, r_O],
                        cell=cell, pbc=True)

In [4]:
job_int = pr.create.job.Lammps("test", delete_existing_job=True)

job_int.structure = water
job_int.interactive_open()
job_int.calc_static()
job_int.run()
job_int.interactive_close()

/cmmc/u/chandu/programs/pyiron_mpie/pyiron_atomistics/pyiron_atomistics/lammps/base.py:210: UserWarning: WARNING: Non-'metal' units are not fully supported. Your calculation should run OK, but results may not be saved in pyiron units.
  "WARNING: Non-'metal' units are not fully supported. Your calculation should run OK, but "
/cmmc/u/chandu/programs/pyiron_mpie/pyiron_atomistics/pyiron_atomistics/lammps/base.py:262: UserWarning: No potential set via job.potential - use default potential, H2O_tip3p
  warnings.warn("No potential set via job.potential - use default potential, " + lst_of_potentials[0])


The job test was saved and received the ID: 15453531


In [5]:
job = pr.create.job.Lammps("test_ni", delete_existing_job=True)
job.structure = water
job.calc_static()
job.run()

The job test_ni was saved and received the ID: 15453533


In [8]:
# Assert that the unit converstions work even in the interactive mode

int_nodes = job_int["output/generic"].list_nodes()
usual_nodes = job["output/generic"].list_nodes()
for node in int_nodes:
    if node in usual_nodes:
        print(node)
        assert np.allclose(job_int["output/generic/" + node], job["output/generic/" + node])

cells
energy_pot
energy_tot
forces
indices
positions
pressures
steps
temperature
volume
